In [1]:
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import words, stopwords

import hunspell
import re, os, io
from tqdm import tqdm
import multiprocessing as mp
num_cores = mp.cpu_count()
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import editdistance

from joblib import Parallel, delayed


In [2]:
f = open('./glove.840B.300d.txt') #Use this as our list of valid words
valid_words = []
for line in f:
    values = line.split()
    word = ' '.join(values[:-300])
    #coefs = np.asarray(values[-300:], dtype='float32')
    valid_words.append(word)
f.close()

In [3]:
spellchecker = hunspell.HunSpell('./index.dic', './index.aff')

In [4]:
f = open("bad-words.txt", "r") #Using a dictionary for duplicates (fuck vs fuk vs f_u_c_k) might be better
badwords = f.read().splitlines()
f.close()

In [5]:
word_set = set(valid_words + badwords) #Much faster searching, add the badwords too.
del(valid_words) #clean up


regex_notascii = re.compile(r'[^\x00-\x7f]')

# changed .* to .*? to make the regex non greedy
regex_underscorelink = re.compile(r'\[(.*?)_link:.*?\]',re.IGNORECASE)
# changed .* to .*? to make the regex non greedy
regex_tags = re.compile(r'\[\[(category|wikipedia|user).*?\]\]')

#added # since some urls have hashtags at the end
regex_url = re.compile(r'''http[s]?://(?:[a-zA-Z]|[0-9]|[#$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+''')
regex_email = re.compile(r'''(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])''')

# added some regex to recove swarwords
regex_dc_shit = re.compile(r's(h|\*)(i|\*)(t|\*)')
regex_dc_fuck = re.compile(r'f(u|\*)(c|\*)(k|\*)')
regex_dc_dick = re.compile(r'd(i|\*)(c|\*)(k|\*)')
regex_dc_ass = re.compile(r'a(s|\*)(s|\*)')
regex_dc_cunt = re.compile(r'c(u|\*)(n|\*)(t|\*)')
regex_dc_bitch = re.compile(r'b(i|\*)(t|\*)(c|\*)(h|\*)')

regex_strayapos = re.compile(r'''((?<=[^a-zA-Z])'|'(?![a-zA-Z]))''')
regex_nums = re.compile(r'\d+')
punct = '''!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~'''

stops = set(stopwords.words("english"))
stemmer = SnowballStemmer('english')


In [12]:
def clean_sentence(dirty_sentence):
    dirty_sentence = dirty_sentence.lower() #Lowercase
    dirty_sentence = regex_underscorelink.sub(' ', dirty_sentence) #Remove everything inside [wiki_link: ...] etc
    dirty_sentence = regex_tags.sub(' ', dirty_sentence) #Remove some extra tags
    dirty_sentence = regex_url.sub(' ', dirty_sentence) #Remove URLs
    dirty_sentence = regex_email.sub(' ', dirty_sentence) #Remove E-mails
    dirty_sentence = regex_nums.sub(' ', dirty_sentence) #Remove numbers    
    dirty_sentence = regex_notascii.sub(' ', dirty_sentence) #Remove some non-ASCII (hunspell has some difficulties otherwise)
    dirty_sentence = regex_strayapos.sub(' ', dirty_sentence) #Remove stray apostrophes
    
    # recovering bad words
    # only applicable in fringe cases ? mabe remove later on
    dirty_sentence = regex_dc_shit.sub('shit', dirty_sentence)
    dirty_sentence = regex_dc_fuck.sub('fuck', dirty_sentence)
    dirty_sentence = regex_dc_dick.sub('dick', dirty_sentence)
    dirty_sentence = regex_dc_ass.sub('ass', dirty_sentence)
    dirty_sentence = regex_dc_cunt.sub('cunt', dirty_sentence)
    dirty_sentence = regex_dc_bitch.sub('bitch', dirty_sentence)
    
    
    dirty_sentence = re.sub(r"[^A-Za-z']", " ", dirty_sentence)
    dirty_sentence = re.sub(r"what's", "what is ", dirty_sentence)
    dirty_sentence = re.sub(r"\'s", " ", dirty_sentence)
    dirty_sentence = re.sub(r"\'ve", " have ", dirty_sentence)
    dirty_sentence = re.sub(r"n't", " not ", dirty_sentence)
    dirty_sentence = re.sub(r"i'm", "i am ", dirty_sentence)
    dirty_sentence = re.sub(r"'m", " am ", dirty_sentence)
    dirty_sentence = re.sub(r"\'re", " are ", dirty_sentence)
    dirty_sentence = re.sub(r"\'d", " would ", dirty_sentence)
    dirty_sentence = re.sub(r"\'ll", " will ", dirty_sentence)
    dirty_sentence = re.sub(r" e g ", " eg ", dirty_sentence)
    dirty_sentence = re.sub(r" b g ", " bg ", dirty_sentence)
    dirty_sentence = re.sub(r" u s ", " american ", dirty_sentence)
    dirty_sentence = re.sub(r"e - mail", "email", dirty_sentence)
    dirty_sentence = re.sub(r"j k", "jk", dirty_sentence)
    dirty_sentence = re.sub(r"\s{2,}", " ", dirty_sentence)
    
    cleaning = word_tokenize(dirty_sentence)
    
    
    wordlist = []
    for word in cleaning:            
        if word in word_set: #If this is a word, add it
            wordlist.append(word)
        else:
            autotry = spellchecker.suggest(word) #Try to correct word
            if len(autotry)==0:
                wordlist.append(word) #No suggestions found, add it as-is
                continue
            
            autocorrect = autotry[0] #Take the first suggestion
            if editdistance.eval(word, autocorrect) >3: #Suggestion looks too different from word, leave it alone and add
                wordlist.append(word)
                continue
            
            wordlist.append(autocorrect) #Replace word with our suggested word
    
    # remove stop words like I, the, for, ...
    #wordlist = [w for w in wordlist if not w in stops]
    
    #stemming words running -> run
    #wordlist = [stemmer.stem(word) for word in wordlist]
    
    return " ".join(wordlist)

In [15]:
clean_sentence("summaries fuck I'm we'll great")

'summaries fuck i am we will great'

In [16]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [17]:
clean_train = ['']*len(list_sentences_train)
for i in tqdm(range(len(list_sentences_train))):
    clean_train[i] = clean_sentence(list_sentences_train[i])
    

train["comment_text"]=clean_train
train.to_csv("train_cleaned_unfiltered.csv", index = False)

100%|██████████| 159571/159571 [56:48<00:00, 46.82it/s]  


In [18]:
clean_test = ['']*len(list_sentences_test)
for i in tqdm(range(len(list_sentences_test))):
    clean_test[i] = clean_sentence(list_sentences_test[i])
    

test["comment_text"]=clean_test
test.to_csv("test_cleaned_unfiltered.csv", index = False)

100%|██████████| 153164/153164 [1:08:09<00:00, 32.96it/s]           
